In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle

In [2]:
data_files_list = ["data/ids2018_processed/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv",
                   "data/ids2018_processed/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv", 
                   "data/ids2018_processed/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv",                   
                   "data/ids2018_processed/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv",
                   "data/ids2018_processed/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv",
                   "data/ids2018_processed/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv"]

save_dir = "data/ids2018_collaborative"

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class AutoEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim, dropout_prob = 0.1):
        super(AutoEncoder, self).__init__()

        self.dropout = nn.Dropout(dropout_prob)
        
        self.encoder = nn.ModuleList([nn.Linear(in_features=input_dim, out_features=latent_dim*2)])
        self.encoder.append(nn.ReLU())
        self.encoder.append(nn.Linear(in_features=latent_dim*2, out_features=latent_dim))
        self.encoder.append(nn.ReLU())

        self.decoder = nn.ModuleList([nn.Linear(in_features=latent_dim, out_features=latent_dim*2)])
        self.decoder.append(nn.ReLU())
        self.decoder.append(nn.Linear(in_features=latent_dim*2, out_features=input_dim))
        self.decoder.append(nn.ReLU())

    def forward(self, x):
        z = self.dropout(x)
        for layer in self.encoder:
            z = layer(z)

        # decoding
        for layer in self.decoder:
            z = layer(z)

        x_hat = z

        return x_hat

In [9]:
def train(csv_path):
    basename = csv_path.split("/")[-1].replace('.csv', '')
    print("\n\n"+basename+"\n"+"="*50)
    # load the training data
    Xy = np.load("{}/{}_train.npy".format(save_dir, basename), allow_pickle=True).astype(float)
    
    # split it to training data and validation data (6:2)
    Xy_train, Xy_valid = train_test_split(Xy, test_size=0.25)
    X_train = Xy_train[:, 1:]
    valid_identifier, X_valid = Xy_valid[:, 0], Xy_valid[:, 1:]

    # normalize the input
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_valid = scaler.transform(X_valid)

    # set the training environment
    model = AutoEncoder(56, 6)
    learning_rate = 0.015
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    cost = nn.MSELoss()
    
    # Training
    X_train = torch.FloatTensor(X_train)
    epochs = 500
    for epoch in tqdm(range(epochs)):
        optimizer.zero_grad()
        X_hat = model(X_train)
        loss = cost(X_train, X_hat)
        loss.backward()
        optimizer.step()
        if epoch%50 == 0:
            print(loss)
            
    # Validating        
    X_valid = torch.FloatTensor(X_valid)
    model.eval()
    X_hat = model(X_valid)
    
    # Get mean loss for each identifier
    valid_losses = ((X_valid-X_hat)**2).mean(dim=1).detach().numpy()
    df = pd.DataFrame(data={'id': valid_identifier, 'loss': valid_losses})
    losses = df.groupby('id').mean().values
    
    # Precompute the alpha thresholds
    total_negative = len(losses)
    alpha = np.arange(1.0, -0.01, -0.05).tolist()
    alpha_i = 0
    alpha_thresholds = [0]*len(alpha)
    for threshold in np.arange(0.0, 2.0005, 0.001):
        fpr = np.sum(losses>threshold)/total_negative
        if (fpr<alpha[alpha_i]):
            alpha_i += 1
        if (fpr>=alpha[alpha_i]):
            alpha_thresholds[alpha_i]=threshold

    print(alpha_thresholds)
    
    # Save the model
    with open("{}/model/{}_scaler.pkl".format(save_dir, basename), 'wb') as f:
        pickle.dump(scaler, f)
    with open("{}/model/{}_thresholds.pkl".format(save_dir, basename), 'wb') as f:
        pickle.dump(alpha_thresholds, f)
    torch.save(model.state_dict(), "{}/model/{}.model".format(save_dir, basename))


In [10]:
for d in data_files_list:
    train(d)



Wednesday-14-02-2018_TrafficForML_CICFlowMeter


  0%|          | 1/500 [00:00<06:10,  1.35it/s]

tensor(0.8295, grad_fn=<MeanBackward0>)


 10%|█         | 51/500 [00:24<03:33,  2.10it/s]

tensor(0.4449, grad_fn=<MeanBackward0>)


...

tensor(0.4137, grad_fn=<MeanBackward0>)


 90%|█████████ | 451/500 [04:26<00:28,  1.71it/s]

tensor(0.4093, grad_fn=<MeanBackward0>)


100%|██████████| 500/500 [04:54<00:00,  1.70it/s]


[0.089, 0.164, 0.186, 0.202, 0.216, 0.229, 0.241, 0.254, 0.267, 0.279, 0.293, 0.307, 0.321, 0.336, 0.354, 0.377, 0.403, 0.446, 0.539, 0.922, 2.0]
